In [3]:
"""
AutoEncoderZ: A Deep Learning Autoencoder for Dimensionality Reduction and Reconstruction
Author: [Zahra Zali]
Date: [July 2024]

Description:
This code implements an advanced autoencoder designed for handling high-dimensional data. 
The autoencoder compresses input data into a compact, low-dimensional feature space and 
reconstructs the original input from these features with high accuracy.

Dependencies:
- NumPy
- TensorFlow/Keras
"""

from numpy.random import seed  
import tensorflow as tf
from keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Reshape, Dense, Concatenate
from keras.models import Model
import keras.backend as K  # Backend for tensor operations

# Set random seeds for reproducibility
sd = 46
seed(sd)  # Seed for NumPy operations
tf.random.set_seed(sd)  # Seed for TensorFlow operations

# Initialize weights using GlorotUniform with a specific seed for reproducibility
initializer = tf.keras.initializers.GlorotUniform(seed=sd)

# **Input Layer**
inp = Input(shape=(320, 512, 1))

# **Encoder: Compresses input to a lower-dimensional representation**

# First convolutional layer: Applies 8 filters with size (7x5), stride (2x2), and ELU activation
e1 = Conv2D(8, (7, 5), strides=[2, 2], activation='elu', 
            kernel_initializer=initializer, padding='same')(inp)

# Second convolutional layer: Applies 16 filters with size (5x3), stride (2x2), and ELU activation
e2 = Conv2D(16, (5, 3), strides=[2, 2], activation='elu', 
            kernel_initializer=initializer, padding='same')(e1)

# Third convolutional layer: Applies 32 filters with size (5x3), stride (1x2)
e3 = Conv2D(32, (5, 3), strides=[1, 2], activation='elu', 
            kernel_initializer=initializer, padding='same')(e2)

# Fourth convolutional layer: Applies 64 filters with size (5x3), stride (1x2)
e4 = Conv2D(64, (5, 3), strides=[1, 2], activation='elu', 
            kernel_initializer=initializer, padding='same')(e3)

# Save shape before flattening for use in the decoder
shape_before_flattening = K.int_shape(e4)

# Flatten the output of the last convolutional layer to create a 1D vector
encoded1 = Flatten()(e4)

# Fully connected layer to encode data into a compact representation
encoded2 = Dense(24, activation='elu')(encoded1)  # Outputs a 24-dimensional feature vector

# Fully connected layer to prepare data for reshaping in the decoder
fc = Dense(163840, activation='elu')(encoded2)  # Matches the total elements needed for reshaping

# **Decoder: Reconstructs the input from the encoded representation**

# Reshape the dense output to match the shape before flattening
d1 = Reshape(shape_before_flattening[1:])(fc)

# First transpose convolutional layer: Expands second dimension by 2x
d2 = Conv2DTranspose(32, (5, 3), strides=[1, 2], activation='elu', 
                     kernel_initializer=initializer, padding='same')(d1)

# Skip connection: Concatenate encoder's output (e3) with the decoder's current output (d2)
c1 = Concatenate()([e3, d2])

# Second transpose convolutional layer: Expands second dimension by 2x
d3 = Conv2DTranspose(16, (5, 3), strides=[1, 2], activation='elu', 
                     kernel_initializer=initializer, padding='same')(c1)

# Skip connection: Concatenate encoder's output (e2) with the decoder's current output (d3)
c3 = Concatenate()([e2, d3])

# Third transpose convolutional layer: Expands both dimensions by 2x
d4 = Conv2DTranspose(8, (5, 3), strides=[2, 2], activation='elu', 
                     kernel_initializer=initializer, padding='same')(c3)

# Skip connection: Concatenate encoder's output (e1) with the decoder's current output (d4)
c4 = Concatenate()([e1, d4])

# Final transpose convolutional layer: Reconstructs the input with 1 channel
decoded = Conv2DTranspose(1, (7, 5), strides=[2, 2], activation='linear', 
                          kernel_initializer=initializer, padding='same')(c4)

# **Model Definitions**
# Autoencoder model: Maps input to reconstructed output
autoencoder = Model(inputs=inp, outputs=decoded, name='autoencoder')

# Encoder model: Maps input to encoded feature representation
encoder = Model(inputs=inp, outputs=encoded2, name='encoder')

# Summarize the autoencoder model
autoencoder.summary()


Model: "autoencoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 320, 512, 1)]        0         []                            
                                                                                                  
 conv2d_8 (Conv2D)           (None, 160, 256, 8)          288       ['input_3[0][0]']             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 80, 128, 16)          1936      ['conv2d_8[0][0]']            
                                                                                                  
 conv2d_10 (Conv2D)          (None, 80, 64, 32)           7712      ['conv2d_9[0][0]']            
                                                                                        